## Explainability von Machine Learning Modellen am Beispiel eines Decision Tree und eines Multilayer Perceptron in Scikit-Learn
Dieses Notebook ist Teil von <a href='https://datenverknoten.de/?p=212' target='_blank'>einem Artikel</a> auf www.datenverknoten.de.
<br>Quelle des verwendeten Datensatzes: https://www.kaggle.com/lirilkumaramal/heart-stroke

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier,plot_tree
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.tree import export_graphviz
from sklearn.model_selection import train_test_split
import graphviz
import matplotlib.pyplot as plt
from itertools import chain, combinations

### Datenvorbereitung
Zunächst werden die Daten geladen und die kategorischen Daten in numerische Werte überführt. In diesem Zusammenhang sei auch auf das Problem der Multikolinearität bei Dummy Variablen -> One Hot Encoding hingewiesen, die z.B. mit Pandas einfach erzeugt werden können hingewiesen (Erklärung z.B. hier: https://amanrai77.medium.com/dummy-variable-trap-9068c3f366fe). Doch auch der hier verwendete Ansatz birgt Tücken. Eine Diskussion findet sich z.B. hier: https://www.analyticsvidhya.com/blog/2020/03/one-hot-encoding-vs-label-encoding-using-scikit-learn/. Dies soll aber nicht primärer Bestandteil sein, es wird der gegebene Ansatz als ideal angenommen.

CSV Datei laden und die id Spalte löschen.

In [ ]:
stroke_raw = pd.read_csv('rawdata/train_strokes.csv').drop(columns=['id'])

Label Encoder Objekt erstellen.

In [ ]:
labelencoder = LabelEncoder()

Zeilen mit na werden entfernt. Die kategorischen Daten werden mit dem Label Encoder in numerische Werte überführt.

In [ ]:
stroke_pre = stroke_raw.copy()
stroke_pre = stroke_pre.dropna()
stroke_pre['gender'] = labelencoder.fit_transform(stroke_pre['gender'])
stroke_pre['ever_married'] = labelencoder.fit_transform(stroke_pre['ever_married'])
stroke_pre['work_type'] = labelencoder.fit_transform(stroke_pre['work_type'])
stroke_pre['Residence_type'] = labelencoder.fit_transform(stroke_pre['Residence_type'])
stroke_pre['smoking_status'] = labelencoder.fit_transform(stroke_pre['smoking_status'])

Es ist zu sehen, dass die Verteilung der beiden Klassen hochgradig unbalanciert ist

In [ ]:
print(list(stroke_pre['stroke']).count(0)) # No stroke
print(list(stroke_pre['stroke']).count(1)) # stroke

Mit dieser Lambda-Funktion wird die kleinste Klasse gefunden und die Anzahl der Instanzen in dieser Klasse wird verwendet, um
Instanzen aus der anderen Klasse zu samplen.

In [ ]:
stroke_pre_2 = stroke_pre.groupby('stroke').apply(lambda x: x.sample(stroke_pre.stroke.value_counts().min()))

Nun sind beide Klassen stroke - Ja,Nein gleichmäßig oft vertreten.

In [ ]:
print(list(stroke_pre_2['stroke']).count(0)) # No stroke
print(list(stroke_pre_2['stroke']).count(1)) # stroke

X enthält nun alle Variablen, die zur Vorhersage genutzt werden sollen. y ist die Variable, die vorhergesagt werden soll.

In [ ]:
X = stroke_pre_2.drop(columns=['stroke'])
y = stroke_pre_2['stroke']

Die Trainings- und Testdaten werden erstellt.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,random_state=2)

### Decision Tree
Im ersten Schritt wird ein Entscheidungsbaum erstellt. Der Baum wird visualisert und auch als PDF ausgegeben. In jedem Knoten steht die Bedingung, die zur Entscheidung führt, ob die linke oder die rechte Kante verfolgt wird.

Das Baumobjekt wird initialisiert.

In [ ]:
clf_tree = DecisionTreeClassifier(random_state=0)

Das Baumobjekt wird auf die Daten gefittet.

In [ ]:
clf_tree_viz = clf_tree.fit(X_train, y_train)

Die Genauigkeit des Klassifikators wird anhand der Testdaten ermittelt.

In [ ]:
clf_tree.score(X_test,y_test)

Ausgabe des Baumes hier im Notebook.

In [ ]:
fig = plt.figure(figsize=(25,20))
_ = plot_tree(clf_tree_viz, 
                   feature_names=X.columns,  
                   class_names='stroke',
                   filled=True)

Ausgabe des Baumes in eine PDF Datei.

In [ ]:
fig.savefig("dttree.pdf", bbox_inches='tight')

### Multilayer Perceptron (MLP) -> Neuronales Netz
Im zweiten Schritt wird ein vollverknüpftes neuronales Netz mit zwei hidden layers mit jeweils fünf Neuronen pro layer trainiert. Die Anzahl der maximalen backpropagation Durchläufe wird auf 2500 begrenzt.

MLP Objekt wird erstellt. Zwei hidden layers mit jeweils fünf Neuronen, vollständig verknüpft.

In [ ]:
clf_mlp = MLPClassifier(random_state=1, max_iter=2500,hidden_layer_sizes=(5, 2))

Die Trainingsdaten werden zum fitten verwendet.

In [ ]:
clf_mlp.fit(X_train, y_train)

Die Genauigkeit wird mit den Testdaten bestimmt.

In [ ]:
clf_mlp.score(X_test,y_test)

Es gibt keine Möglichkeit, die Entscheidungsfindung zu visualisieren. Zwar lassen sich die Gewichtungen der einzelnen Neuronen exportieren, doch diese sind nicht intuitiv zu interpretieren. Darum werden alle Kombinationen der Features erzeugt und es wird jedes mal das Netz mit dieser Kombination trainiert. Anhand der Genauigkeit wird dann abgelesen, welche Kombination die beste Vorhersage ergibt. So lässt sich zumindest erkennen, welche Features vom MLP als wichtig angesehen werden.

Diese Funktion erzeugt alle möglichen Kombinationen der Features

In [ ]:
def powerset(iterable):
    xs = list(iterable)
    return chain.from_iterable(combinations(xs,n) for n in range(len(xs)+1))

Hier wird eine Liste aller möglichen Kombinationen mit der zuvor definierten Funktion erstellt.

In [ ]:
column_combinations = []
for s in powerset(list(stroke_raw.drop(columns=['stroke']).columns)):
    if(len(list(s))>0):
        column_combinations.append(list(s))

Diese Kombinationen werden in einem MLP Objekt genutzt. Die Genauigkeit und die jeweilige Kombination wird in einem DataFrame gesammelt.

In [ ]:
result_frame = pd.DataFrame(columns=['Features','Featurelist','Accuracy'])

for combination in column_combinations:
    clf_mlp = MLPClassifier(random_state=1, max_iter=2500,hidden_layer_sizes=(5, 2))
    clf_mlp.fit(X_train[combination], y_train)
    score = clf_mlp.score(X_test[combination],y_test)
    result_frame = result_frame.append({\
                                        'Features':len(combination),\
                                        'Featurelist':combination,\
                                        'Accuracy':score
                                       },ignore_index=True)

Der DataFrame wird nach Genauigkeit sortiert. Die Featurekombination mit der höchsten Genauigkeit ist als diejenige anzunehmen, die am meisten zu einer Aussage über stroke - Ja,Nein beiträgt.

In [ ]:
result_frame.sort_values(by = 'Accuracy',ascending = False, inplace = True)

An erster Stelle sind die Features Geschlecht, Alter, Bluthochdruck und Status Raucher (Ja,Nein) als Features zu erkennen, die die höchste Genauigkeit erzeugen.

In [ ]:
result_frame.head(5)

### Das Subset Problem
Wird das Sampling mehrfach ausgeführt, zeigt sich, dass sowohl die höchste Genauigkeit als auch die zugehörige Featureliste variiert. Dies hängt mit dem Subset zusammen, das für die dominante Klasse gewählt wird. Um dies zu verdeutlichen, wird das oben beschriebene Beispiel zum Multilayer Perceptron mehrfach ausgeführt. Dabei wird sowohl jedes mal das Sampling als auch das Erstellen eines Trainings- und Testdatensatzes neu ausgeführt. Nur das Ergebnis mit der höchsten Genauigkeit wird in der Ergebnistabelle gespeichert.

In [ ]:
overall_resultframe = pd.DataFrame(columns = ['Run','Features','Featurelist','Accuracy'])

for i in range(0,25):
    print("Durchlauf: "+str(i))
    stroke_pre_2 = stroke_pre.groupby('stroke').apply(lambda x: x.sample(stroke_pre.stroke.value_counts().min()))
    
    X = stroke_pre_2.drop(columns=['stroke'])
    y = stroke_pre_2['stroke']
    # Diesmal wird das Trainings- und Testset ohne random_state erstellt, sodass die Wahl der Instanzen variiert. 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
    
    clf_mlp = MLPClassifier(random_state=1, max_iter=2500,hidden_layer_sizes=(5, 2))
    _ = clf_mlp.fit(X_train, y_train)
    
    result_frame = pd.DataFrame(columns=['Features','Featurelist','Accuracy'])

    # Die Kombinationen wurden bereits erstellt und bleiben bestehen.
    for combination in column_combinations:
        clf_mlp = MLPClassifier(random_state=1, max_iter=2500,hidden_layer_sizes=(5, 2))
        clf_mlp.fit(X_train[combination], y_train)
        score = clf_mlp.score(X_test[combination],y_test)
        result_frame = result_frame.append({\
                                            'Features':len(combination),\
                                            'Featurelist':combination,\
                                            'Accuracy':score
                                           },ignore_index=True)
    
    # Die Ergebnisse werden im overall_resultframe gespeichert.
    result_frame.sort_values(by = 'Accuracy',ascending = False, inplace = True)
    best = result_frame.head(1).to_dict(orient='records')
    best[0]['Run'] = i
    overall_resultframe = overall_resultframe.append(best[0],ignore_index = True)

Allein in den ersten zehn Durchläufen lässt sich erkennen, dass die ausgewählten Features und die Genauigkeit bei unterschiedlicher Zusammensetzung des Trainings- und Testdatensatzes schwanken.

In [ ]:
overall_resultframe.head(10)